## doc ds



In [1]:
from langchain_core.documents import Document

In [2]:
doc=Document(
    page_content="this is main text conetent used for RAG", 
    metadata={
        "source": "example.pdf", 
        "page": 1, 
        "other_info": "additional metadata"})

doc

Document(metadata={'source': 'example.pdf', 'page': 1, 'other_info': 'additional metadata'}, page_content='this is main text conetent used for RAG')

In [3]:
import os
os.makedirs("../data/text_files", exist_ok=True)

In [4]:
# from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

In [5]:
loader=TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
loader.load()

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Introduction to Python\n\nPython is a high-level, interpreted programming language known for its simplicity and readability. It emphasizes clear syntax, which makes it beginner-friendly yet powerful for advanced programming. Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming.\n\nKey Features\n\nEasy to Read and Write: Simple syntax similar to English.\n\nInterpreted: Executes code line by line, making debugging easier.\n\nCross-Platform: Runs on Windows, macOS, Linux, and more.\n\nExtensive Libraries: Offers libraries like NumPy, Pandas, Matplotlib, TensorFlow, and Flask.\n\nOpen Source: Free to use with a large developer community.\n\nCommon Uses\n\nWeb Development: Using frameworks like Django and Flask.\n\nData Science: For data analysis, visualization, and machine learning.\n\nAutomation: Writing scripts to automate repetitive tasks.\n\n

In [6]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf-8"},
    show_progress=True
    )


text_documents=dir_loader.load()
print(text_documents)

100%|██████████| 2/2 [00:00<?, ?it/s]

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Introduction to Machine Learning\n\nMachine learning (ML) is a subset of artificial intelligence (AI) that enables systems to learn from data and improve performance on a task without being explicitly programmed. Instead of following hard-coded rules, machine learning algorithms detect patterns in data and make predictions or decisions based on them.\n\nTypes of Machine Learning\n\nSupervised Learning\nIn supervised learning, the algorithm is trained on labeled data, meaning the input comes with the correct output. The goal is for the model to learn the mapping between inputs and outputs to predict outcomes for new data.\n\nExamples: Predicting house prices, classifying emails as spam or not, forecasting sales.\n\nCommon Algorithms: Linear regression, decision trees, support vector machines, neural networks.\n\nUnsupervised Learning\nUnsupervised learning deals with unlabeled data. The algorithm 

## load all pdf

In [7]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

dir_loader=DirectoryLoader(
    "../data/pdf",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,
    # loader_kwargs={"encoding":"utf-8"},
    show_progress=True
    )

pdf_documents=dir_loader.load()
pdf_documents

100%|██████████| 5/5 [00:14<00:00,  2.88s/it]


[Document(metadata={'producer': 'pdfTeX-1.40.22', 'creator': 'LaTeX with Beamer class', 'creationdate': '2023-10-14T23:13:47+02:00', 'author': 'Myra Spiliopoulou', 'title': "DM I: Block 'Classification' - Unit 'Decision Trees'", 'subject': '', 'keywords': '', 'moddate': '2023-10-14T23:13:47+02:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'source': '..\\data\\pdf\\1_Classification_Unit_DTs.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}, page_content='DM I: Block ’Classification’\nUnit ’Decision Trees’\nMyra Spiliopoulou'),
 Document(metadata={'producer': 'pdfTeX-1.40.22', 'creator': 'LaTeX with Beamer class', 'creationdate': '2023-10-14T23:13:47+02:00', 'author': 'Myra Spiliopoulou', 'title': "DM I: Block 'Classification' - Unit 'Decision Trees'", 'subject': '', 'keywords': '', 'moddate': '2023-10-14T23:13:47+02:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Ve

In [8]:
type(pdf_documents)

list

## Split into chucks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, # Each chunk: ~1000 characters
        chunk_overlap=chunk_overlap, # 200 chars overlap for context
        length_function=len, # How to measure length
        separators=["\n\n", "\n", " ", ""] # Split hierarchy
    )
    # Actually split the documents
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show what a chunk looks like
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [17]:
chunks= split_documents(pdf_documents,chunk_size=1000,chunk_overlap=200)

Split 224 documents into 239 chunks

Example chunk:
Content: DM I: Block ’Classification’
Unit ’Decision Trees’
Myra Spiliopoulou...
Metadata: {'producer': 'pdfTeX-1.40.22', 'creator': 'LaTeX with Beamer class', 'creationdate': '2023-10-14T23:13:47+02:00', 'author': 'Myra Spiliopoulou', 'title': "DM I: Block 'Classification' - Unit 'Decision Trees'", 'subject': '', 'keywords': '', 'moddate': '2023-10-14T23:13:47+02:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'source': '..\\data\\pdf\\1_Classification_Unit_DTs.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}


## embedding and vector store

In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer   
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
import sys
print(sys.executable)
from sentence_transformers import SentenceTransformer
import chromadb
print("Imports OK: sentence_transformers and chromadb available")

c:\Program Files\Python311\python.exe
Imports OK: sentence_transformers and chromadb available


In [ ]:
class EmbeddingManager:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        
        self.model_name = model_name
        self.model =None
        self.load_model()#load teh model during initialization
    
    def load_model(self):       
        try:
            print(f"Loading model '{self.model_name}'...")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model '{self.model_name}' loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
            #convert text to dimension 384
        except Exception as e:
            print(f"Error loading model '{self.model_name}': {e}")
            raise e
    def generate_embedding(self, texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model not loaded. Call load_model() first.")
        
        print(f"Generating embedding for text of length {len(texts)}...")
        embedding = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embedding of shape {embedding.shape}.")
        return embedding
   
embedding_manager = EmbeddingManager()
embedding_manager

Loading model 'all-MiniLM-L6-v2'...
Model 'all-MiniLM-L6-v2' loaded successfully. Embedding dimension: 384


## Vectore store

In [14]:
class VectorStore:
    def __init__(self, collecion_name: str = "pdf_documents",persist_directory: str="../data/vector_store"):
    
        self.collecion_name = collecion_name
        self.persist_directory = persist_directory
        self.client=None
        self.collection=None
        self._initialize_store()
        
    def _initialize_store(self):
        try:
            #create directory if not exists
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            
            #where to store the vectors inisde vector db like in firestore
            self.collection = self.client.get_or_create_collection(
                name=self.collecion_name,
                metadata={"description": "PDF document embeddings for RAG"}
                )
            print(f"Vector store initialized at '{self.persist_directory}' with collection '{self.collecion_name}'.")
            print(f"Current number of vectors in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise e
        
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents and embeddings must match.")
        
        ids=[]
        metadatas=[]
        documents_texts=[]
        embeddings_list=[]
        
        for i, (doc , embedding) in enumerate(zip(documents, embeddings)):
            doc_id = str(uuid.uuid4())
            ids.append(doc_id)
            
            metadata=dict(doc.metadata)
            metadata["doc_index"]=i
            metadata["context_length"]=len(doc.page_content)
            metadatas.append(doc.metadata)
            
            
            documents_texts.append(doc.page_content)
            
            
            embeddings_list.append(embedding.tolist())
            
            
        try:
            self.collection.add(
                ids=ids,
                metadatas=metadatas,
                documents=documents_texts,
                embeddings=embeddings_list
            )
            print(f"Added {len(documents)} documents to vector store.")
            print(f"New number of vectors in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise e
        
vector_store = VectorStore()
vector_store
            
    
   

Vector store initialized at '../data/vector_store' with collection 'pdf_documents'.
Current number of vectors in collection: 0


In [18]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.22', 'creator': 'LaTeX with Beamer class', 'creationdate': '2023-10-14T23:13:47+02:00', 'author': 'Myra Spiliopoulou', 'title': "DM I: Block 'Classification' - Unit 'Decision Trees'", 'subject': '', 'keywords': '', 'moddate': '2023-10-14T23:13:47+02:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'source': '..\\data\\pdf\\1_Classification_Unit_DTs.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}, page_content='DM I: Block ’Classification’\nUnit ’Decision Trees’\nMyra Spiliopoulou'),
 Document(metadata={'producer': 'pdfTeX-1.40.22', 'creator': 'LaTeX with Beamer class', 'creationdate': '2023-10-14T23:13:47+02:00', 'author': 'Myra Spiliopoulou', 'title': "DM I: Block 'Classification' - Unit 'Decision Trees'", 'subject': '', 'keywords': '', 'moddate': '2023-10-14T23:13:47+02:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Ve

In [23]:
text = [doc.page_content for doc in chunks]


embedding=embedding_manager.generate_embedding(text)
embedding.shape

vector_store.add_documents(chunks,embedding)

Generating embedding for text of length 239...


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Generated embedding of shape (239, 384).
Added 239 documents to vector store.
New number of vectors in collection: 239


## Retrieval from Vectore store

In [ ]:
class RAGRetriever:
    def __init__(self,vector_store:VectorStore,embedding_manager:EmbeddingManager):
        self.vector_store=vector_store
        self.embedding_manager=embedding_manager
        
    def retrieve(self,query:str,top_k:int=5,score_threshold:float=0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents from the vector store based on the query.
        
        Parameters:
        - query: The input query string.
        - top_k: Number of top similar documents to retrieve.
        - score_threshold: Minimum similarity score to consider a document relevant.
        
        Returns:
        - List of dictionaries containing 'document' and 'score'.
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score Threshold: {score_threshold}")
        
        # Generate embedding for the query from user
        query_embedding = self.embedding_manager.generate_embedding([query])[0]
        
        # Perform similarity search in the vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
        
            retrieved_docs = []
            if results['documents'] and results['documents'][0]:
                documents=results['documents'][0]
                metadatas=results['metadatas'][0]
                distances=results['distances'][0]
                ids=results['ids'][0]
                
                for i, (doc_id,document, metadata, distance) in enumerate(zip(ids,documents,metadatas,distances)):
                    similarity_score=1 - distance
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            "id": doc_id,
                            "content": document,
                            "metadata": metadata,
                            "similarity_score": similarity_score,
                            "distance": distance,
                            "rank": i + 1
                        })
                print(f"Retrieved {len(retrieved_docs)} documents above the score threshold.")
            else:
                print("No documents retrieved from vector store.")
            return retrieved_docs
        except Exception as e:
            print(f"Error during retrieval: {e}")
            raise e
rag_retriever=RAGRetriever(vector_store,embedding_manager)   
            
         

In [26]:
rag_retriever

In [27]:
rag_retriever.retrieve("What is Naive Bayes?")

Retrieving documents for query: 'What is Naive Bayes?'
Top K: 5, Score Threshold: 0.0
Generating embedding for text of length 1...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embedding of shape (1, 384).
Retrieved 5 documents above the score threshold.


[{'id': 'f3829e6d-85de-4004-9f39-3cf5808160ba',
  'content': 'DM I: Block ’Classification’\nUnit ’Naive Bayes’\nMyra Spiliopoulou',
  'metadata': {'creator': 'LaTeX with Beamer class',
   'keywords': '',
   'subject': '',
   'page_label': '1',
   'title': "DM I: Block 'Classification' - Unit 'Naive Bayes'",
   'source': '..\\data\\pdf\\1_Classification_Unit_NB.pdf',
   'author': 'Myra Spiliopoulou',
   'page': 0,
   'moddate': '2023-10-07T20:24:48+02:00',
   'trapped': '/False',
   'producer': 'pdfTeX-1.40.22',
   'total_pages': 23,
   'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev',
   'creationdate': '2023-10-07T20:24:48+02:00'},
  'similarity_score': 0.3949319124221802,
  'distance': 0.6050680875778198,
  'rank': 1},
 {'id': '096c27d3-fe93-4e28-8219-7bf7df125b42',
  'content': 'The Law of Bayes for Model Learning The Zero-Frequency Problem NB on numerical attributes Closing\nClassification with Naive Bayes fr